In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

In [5]:
def ColumnsExtractor(data):
    reduced_data = data[data["AMT", "TRANS_MONTH", "TRANS_DAY", "TRANS_HOUR", "CATEGORY", "IS_FRAUD"]]
    reduced_data = reduced_data["IS_FRAUD"].astype("int")
    reduced_data.to_csv("reduced_data.csv",index=False) #디버깅용
    return reduced_data

In [15]:
def KNNforFraud(df, isLearn, beta, delta1, epsilon):
    if(not isLearn):
        #학습이 아닌 추론일 때만 사용됨
        df_train_x = pd.read_csv("../data/splitted/Fraud_Detection_train_features.csv", index_col=0)
    
    reduced_data = ColumnsExtractor(df)
    
    #MinMaxScaler 함수로 객체만들고 scaled된 reduced_data_scaled 만들기
    MMscaler = MinMaxScaler()
    if(isLearn):
        MMscaler.fit(reduced_data)
        reduced_data_scaled = MMscaler.transfrom(reduced_data)
        reduced_data_scaled = pd.DataFrame(reduced_data_scaled, columns=["AMT", "TRANS_MONTH", "TRANS_DAY", "TRANS_HOUR", "CATEGORY", "IS_FRAUD"])
    else:
        #학습용 데이터 맨 아래에 임시로 행을 붙인 뒤 다시 뗀다
        df_temp = pd.concat([df_train_x, reduced_data])
        MMscaler.fit(df_temp)
        df_temp2 = MMscaler.transfrom(df_temp)
        df_temp2 = pd.DataFrame(df_temp2, columns=["AMT", "TRANS_MONTH", "TRANS_DAY", "TRANS_HOUR", "CATEGORY", "IS_FRAUD"])
        reduced_data_scaled = df_temp2.tail(1)

    #5개의 KNN 객체 생성. parameter는 반드시 odd number로!
    global kn1
    global kn2
    global kn3
    global kn4
    global kn5
    
    global FCmodel
    
    if(isLearn):
        kn1 = KNeighborsClassifier(n_neighbors=3)
        kn2 = KNeighborsClassifier(n_neighbors=3)
        kn3 = KNeighborsClassifier(n_neighbors=3)
        kn4 = KNeighborsClassifier(n_neighbors=3)
        kn5 = KNeighborsClassifier(n_neighbors=3)
        
        kn1.fit(reduced_data_scaled["AMT"], reduced_data_scaled["IS_FRAUD"])
        kn2.fit(reduced_data_scaled["TRANS_MONTH"], reduced_data_scaled["IS_FRAUD"])
        kn3.fit(reduced_data_scaled["TRANS_DAY"], reduced_data_scaled["IS_FRAUD"])
        kn4.fit(reduced_data_scaled["TRANS_HOUR"], reduced_data_scaled["IS_FRAUD"])
        kn5.fit(reduced_data_scaled["CATEGORY"], reduced_data_scaled["IS_FRAUD"])
        arr1 = kn1.predict_proba(reduced_data_scaled["AMT"])
        arr2 = kn2.predict_proba(reduced_data_scaled["TRANS_MONTH"])
        arr3 = kn3.predict_proba(reduced_data_scaled["TRANS_DAY"])
        arr4 = kn4.predict_proba(reduced_data_scaled["TRANS_HOUR"])
        arr5 = kn5.predict_proba(reduced_data_scaled["CATEGORY"])
        
        #arr1~5를 하나의 numpy 배열로 합칠 것이다. 하나의 데이터는 하나의 행이다.
        NNinput = arr1[:,1]
        NNinput = np.append(NNinput, arr2[:,1], axis=1)
        NNinput = np.append(NNinput, arr3[:,1], axis=1)
        NNinput = np.append(NNinput, arr4[:,1], axis=1)
        NNinput = np.append(NNinput, arr5[:,1], axis=1)
        NNoutput = np.array(reduced_data_scaled["IS_FRAUD"].to_list())
        NNoutput = NNoutput.T
        
        #Fully Connected층을 epoch = epsilon횟수로 학습
        FCmodel = keras.Sequential()
        FCmodel.add(keras.layers.Dense(beta))
        FCmodel.add(keras.layers.Dense(1, activation="sigmoid"))
        FCmodel.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")
        FCmodel.fit(NNinput, NNoutput, epochs=epsilon)
        return 2
    else:
        arr1 = kn1.predict_proba(reduced_data_scaled["AMT"])
        arr2 = kn2.predict_proba(reduced_data_scaled["TRANS_MONTH"])
        arr3 = kn3.predict_proba(reduced_data_scaled["TRANS_DAY"])
        arr4 = kn4.predict_proba(reduced_data_scaled["TRANS_HOUR"])
        arr5 = kn5.predict_proba(reduced_data_scaled["CATEGORY"])
        
        arr1 = arr1[:,1]
        arr2 = arr2[:,1]
        arr3 = arr3[:,1]
        arr4 = arr4[:,1]
        arr5 = arr5[:,1]
        
        arr_input = arr1
        arr_input = np.append(arr_input, arr2)
        arr_input = np.append(arr_input, arr3)
        arr_input = np.append(arr_input, arr4)
        arr_input = np.append(arr_input, arr5)
        
        fraud_prob = FCmodel.predict(arr_input)
        return fraud_prob